In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ======================== 配置（区分强制筛选和排序指标） ========================
CONFIG = {
    "factortable_path": r'D:\workspace\xiaoyao\data\factortable.parquet',
    "target_date": "2025-10-23",
    "result_save_path": r'./single_day_selection_20251010.csv',
    "log_path": r'./single_day_selection_log.txt',
    "top_n": 20,  # 最终选出Top20股票
    # 强制筛选条件（核心逻辑，必须满足）
    "filter_thresholds": {
        "rise_ratio_30d_min": 0.1,      # 30日涨幅≥10%
        "pullback_ratio_min": 0.05,     # 回调幅度≥5%
        "pullback_ratio_max": 0.3,      # 回调幅度≤30%
        "pullback_days_min": 5,         # 回调天数≥5天
        "pullback_days_max": 30,        # 回调天数≤30天
        "close_ge_ma20": True           # 收盘价≥MA20
    },
    # 排序指标权重（总和为1，权重越高越重要）
    "sort_weights": {
        "volume_ratio_5d": 0.2,         # 量能强度（越高越好）
        "rsi_score": 0.15,              # RSI位置得分（接近30分越高）
        "bollinger_score": 0.2,         # 布林带接近度（越近下轨得分越高）
        "macd_score": 0.25,             # MACD动能得分（越高越好）
        "price_rise_score": 0.2         # 价格抬升得分（连续上涨天数越多越高）
    }
}

# ======================== 工具函数 ========================
def init_log():
    if os.path.exists(CONFIG["log_path"]):
        os.remove(CONFIG["log_path"])
    log_msg(f"启动 {CONFIG['target_date']} 选股（强制筛选+柔性排序）")

def log_msg(msg):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_line = f"[{current_time}] {msg}"
    print(log_line)
    with open(CONFIG["log_path"], 'a', encoding='utf-8') as f:
        f.write(log_line + "\n")

def validate_data(df):
    """验证必需字段，确保'close'存在"""
    required_cols = ['stock_code', 'date', 'close', 'ma20', 'high_30d', 
                    'pullback_ratio', 'pullback_days', 'volume_ratio_5d', 
                    'bollinger_lower', 'rsi14', 'macd_line', 'signal_line', 
                    'macd_hist']
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"字段缺失：{missing}，请检查factortable数据")
    log_msg("✅ 数据字段验证通过（含'close'列）")
    return df

# ======================== 步骤1：计算衍生指标（修复KeyError） ========================
def pre_calculate_indicators(df):
    # 1. 计算30日涨幅（修复：传递DataFrame，避免Series访问错误）
    def calc_rise_ratio(group):
        group['rise_ratio_30d'] = (group['high_30d'] - group['close'].shift(30)) / \
                                 group['close'].shift(30).replace(0, 0.0001)
        return group['rise_ratio_30d']
    # 关键修复：传递包含'close'和'high_30d'的DataFrame，而非单独Series
    df['rise_ratio_30d'] = df.groupby('stock_code', group_keys=False)[['close', 'high_30d']].apply(calc_rise_ratio)
    df['rise_ratio_30d'] = df['rise_ratio_30d'].fillna(0)
    
    # 2. 计算连续上涨天数（核心修复：处理Series输入，避免group['close']访问）
    def calc_up_days(close_series):
        """参数为'close'列的Series，直接用series计算"""
        up = close_series > close_series.shift(1)
        # 连续上涨天数：上涨则累加，下跌则重置
        consecutive_up = up.groupby(up.ne(up.shift()).cumsum()).cumsum()
        return consecutive_up
    # 传递'close'列Series，函数内直接使用，不访问列名
    df['consecutive_up_days'] = df.groupby('stock_code', group_keys=False)['close'].apply(calc_up_days)
    
    log_msg("✅ 衍生指标计算完成（30日涨幅+连续上涨天数，已修复KeyError）")
    return df

# ======================== 步骤2：强制筛选（保留基础股票池） ========================
def filter_stocks(daily_df):
    thresholds = CONFIG["filter_thresholds"]
    # 强制条件（必须同时满足）
    conds = [
        daily_df['rise_ratio_30d'] >= thresholds['rise_ratio_30d_min'],
        daily_df['pullback_ratio'].between(thresholds['pullback_ratio_min'], thresholds['pullback_ratio_max']),
        daily_df['pullback_days'].between(thresholds['pullback_days_min'], thresholds['pullback_days_max']),
        daily_df['close'] >= daily_df['ma20']  # 站稳MA20
    ]
    filtered_df = daily_df[np.all(conds, axis=0)].copy()
    log_msg(f"强制筛选后股票池大小：{len(filtered_df)}/{len(daily_df)}")
    if len(filtered_df) == 0:
        raise ValueError("强制筛选后无股票，建议放宽filter_thresholds（如30日涨幅≥5%）")
    return filtered_df

# ======================== 步骤3：柔性排序（对筛选后股票打分） ========================
def score_stocks(filtered_df):
    """将排序指标标准化为0-100分，按权重计算综合得分"""
    df = filtered_df.copy()
    weights = CONFIG["sort_weights"]
    
    # 1. 量能强度得分（volume_ratio_5d越高得分越高，标准化为0-100）
    vol_min, vol_max = df['volume_ratio_5d'].min(), df['volume_ratio_5d'].max()
    df['volume_score'] = ((df['volume_ratio_5d'] - vol_min) / (vol_max - vol_min + 1e-8)) * 100
    
    # 2. RSI位置得分（越接近30分越高，偏离则扣分）
    df['rsi_score'] = 100 - (abs(df['rsi14'] - 30) / 30) * 100  # 30为理想值
    df['rsi_score'] = df['rsi_score'].clip(0, 100)  # 限制在0-100
    
    # 3. 布林带接近度得分（越接近下轨得分越高）
    bollinger_ratio = df['close'] / df['bollinger_lower']  # 比值越小越接近下轨
    df['bollinger_score'] = (1.1 - bollinger_ratio) / (1.1 - 1.0 + 1e-8) * 100  # 1.0-1.1为有效范围
    df['bollinger_score'] = df['bollinger_score'].clip(0, 100)  # 超出范围得0分
    
    # 4. MACD动能得分（柱状图越高+接近金叉得分越高）
    # MACD柱状图得分（0-50分）
    macd_hist_min, macd_hist_max = df['macd_hist'].min(), df['macd_hist'].max()
    df['macd_hist_score'] = ((df['macd_hist'] - macd_hist_min) / (macd_hist_max - macd_hist_min + 1e-8)) * 50
    # MACD接近金叉得分（0-50分）
    macd_diff = df['macd_line'] - df['signal_line']  # 差值越大越接近金叉
    df['macd_cross_score'] = ((macd_diff + 0.1) / (0.1 + 0.1 + 1e-8)) * 50  # 容忍度±0.1
    df['macd_score'] = (df['macd_hist_score'] + df['macd_cross_score']).clip(0, 100)
    
    # 5. 价格抬升得分（连续上涨天数越多得分越高）
    up_days_max = df['consecutive_up_days'].max()
    df['price_rise_score'] = (df['consecutive_up_days'] / (up_days_max + 1e-8)) * 100
    
    # 计算综合得分（按权重加权）
    df['total_score'] = (
        df['volume_score'] * weights['volume_ratio_5d'] +
        df['rsi_score'] * weights['rsi_score'] +
        df['bollinger_score'] * weights['bollinger_score'] +
        df['macd_score'] * weights['macd_score'] +
        df['price_rise_score'] * weights['price_rise_score']
    )
    
    # 按综合得分降序排序
    df = df.sort_values(by='total_score', ascending=False).reset_index(drop=True)
    log_msg(f"柔性排序完成，最高得分：{df['total_score'].max():.2f}，最低得分：{df['total_score'].min():.2f}")
    return df

# ======================== 主流程 ========================
def select_single_day():
    try:
        init_log()
        target_date = pd.to_datetime(CONFIG["target_date"]).date()
        log_msg(f"目标日期：{target_date}，加载数据中...")

        # 1. 加载数据并验证（确保'close'存在）
        df = pd.read_parquet(CONFIG["factortable_path"])
        df['date'] = pd.to_datetime(df['date']).dt.date
        df = validate_data(df)  # 验证包含'close'在内的必需字段

        # 2. 计算衍生指标（修复KeyError）
        df = pre_calculate_indicators(df)

        # 3. 筛选目标日期数据
        daily_df = df[df['date'] == target_date].copy()
        if len(daily_df) == 0:
            raise ValueError(f"无{target_date}数据，可能是休市或数据缺失")
        log_msg(f"原始数据股票数：{len(daily_df)}")

        # 4. 强制筛选（保留基础股票池）
        filtered_df = filter_stocks(daily_df)

        # 5. 柔性排序（打分并选出Top N）
        ranked_df = score_stocks(filtered_df)
        top_df = ranked_df.head(CONFIG["top_n"]).copy()

        # 6. 整理结果（保留核心字段+得分）
        result_cols = [
            'stock_code', 'close', 'rise_ratio_30d', 'pullback_ratio',
            'pullback_days', 'volume_ratio_5d', 'rsi14', 'total_score',
            'consecutive_up_days', 'macd_hist'
        ]
        top_df = top_df[result_cols].reset_index(drop=True)

        # 7. 保存与输出
        top_df.to_csv(CONFIG["result_save_path"], index=False, encoding='utf-8-sig')
        log_msg(f"\n✅ 选股完成：从{len(filtered_df)}只筛选股票中选出Top{CONFIG['top_n']}")
        log_msg(f"结果路径：{CONFIG['result_save_path']}")
        
        print("\nTop10股票列表：")
        print(top_df[['stock_code', 'close', 'total_score', 'rise_ratio_30d', 'pullback_ratio']].head(10).to_string(index=False))

        return top_df

    except Exception as e:
        log_msg(f"❌ 错误：{str(e)}")
        raise

# ======================== 执行 ========================
if __name__ == "__main__":
    select_single_day()

[2025-10-24 09:30:08] 启动 2025-10-10 选股（强制筛选+柔性排序）
[2025-10-24 09:30:08] 目标日期：2025-10-10，加载数据中...
[2025-10-24 09:30:08] ✅ 数据字段验证通过（含'close'列）
[2025-10-24 09:30:16] ✅ 衍生指标计算完成（30日涨幅+连续上涨天数，已修复KeyError）
[2025-10-24 09:30:16] 原始数据股票数：5159
[2025-10-24 09:30:16] 强制筛选后股票池大小：276/5159
[2025-10-24 09:30:16] 柔性排序完成，最高得分：54.07，最低得分：1.49
[2025-10-24 09:30:16] 
✅ 选股完成：从276只筛选股票中选出Top20
[2025-10-24 09:30:16] 结果路径：./single_day_selection_20251010.csv

Top10股票列表：
 stock_code  close  total_score  rise_ratio_30d  pullback_ratio
000757.XSHE  29.83    54.068646        0.273299        0.078752
600512.XSHG  16.69    44.847610        0.121460        0.063412
600207.XSHG   8.55    44.084792        0.176322        0.084582
001221.XSHE  65.29    42.965976        0.286081        0.070208
300563.XSHE  85.90    41.494190        0.173288        0.115891
300341.XSHE  81.06    39.471008        0.121113        0.055355
002453.XSHE  29.95    38.150719        0.510662        0.258296
000921.XSHE  40.92    38.010184       